In [1]:
import pandas as pd
import os
import torch

device = ("cuda" if torch.cuda.is_available() else "cpu")

df = pd.DataFrame(columns=["img_name","label"])
df["img_name"] = os.listdir("Caras_64x/")
for idx, i in enumerate(os.listdir("Caras_64x/")):
    if "Gryffindor" in i:
        df["label"][idx] = [1,0,0,0]
    elif "Hufflepuff" in i:
        df["label"][idx] = [0,1,0,0]
    elif "Ravenclaw" in i:
        df["label"][idx] = [0,0,1,0]
    elif "Slytherin" in i:
        df["label"][idx] = [0,0,0,1]

df.to_csv (r'all_caras.csv', index = False, header=True)

In [2]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class HousesDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("L")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [3]:
import torch.nn as nn
import torchvision.models as models

class CNN(nn.Module):
    def __init__(self, train_CNN=False, num_classes=4):
        super(CNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.inception(images)
        return self.sigmoid(self.dropout(self.relu(features))).squeeze(1)

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
transform = transforms.Compose(
        [   transforms.ToTensor(),
            transforms.Normalize((0.5), (0.5))
        ]
    )

In [6]:
num_epochs = 10
learning_rate = 0.00001
train_CNN = False
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 1

In [7]:
dataset_all = HousesDataset("all","pics_by_house64x.csv",transform=transform)
test_set, train_and_val_set = torch.utils.data.random_split(dataset_all,[40,352])
train_set, validation_set = torch.utils.data.random_split(train_and_val_set,[313,39])

train_loader = DataLoader(dataset=train_set, shuffle=shuffle,
                          batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)

validation_loader = DataLoader(dataset=validation_set, 
                               shuffle=shuffle, batch_size=batch_size,num_workers=num_workers, pin_memory=pin_memory)

In [8]:
model = CNN().to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for name, param in model.inception.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN

In [9]:
def check_accuracy(loader, model):
    if loader == train_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on validation data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            predictions = torch.tensor([np.rint(i).astype(int) for i in scores]).to(device)            
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    return f"{float(num_correct)/float(num_samples)*100:.2f}"
    print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    model.train()

In [10]:
def train():
    model.train()
    for epoch in range(num_epochs):
        loop = tqdm(train_loader, total = len(train_loader), leave = True)
        if epoch % 2 == 0:
            loop.set_postfix(val_acc = check_accuracy(validation_loader, model))
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
            loop.set_postfix(loss = loss.item())

In [ ]:
train()

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Checking accuracy on validation data
